In [1]:
import pandas as pd
from joypy import joyplot
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import numpy as np

In [2]:
%matplotlib inline

In [3]:
samplesubset = 10000

In [4]:
# read in samples
coresamples = pd.read_csv('../input_data/coressamples.csv', index_col=0)
genomescalesamples = pd.read_csv('../input_data/genomescalesamples.csv', index_col=0)

In [5]:
# fix CO2 reaction name for core samples
coresamplecolumnnames = list(coresamples.columns)
coresamplecolumnnames[coresamplecolumnnames.index('co2_exchange')] = 'EX_co2_e'
coresamples.columns = coresamplecolumnnames

In [6]:
# get random subset
randomindices = np.random.choice(coresamples.shape[0], size=samplesubset, replace=False)

In [7]:
# sort reactions by median and break into two subsets
reactionNames = list(pd.DataFrame.median(coresamples, axis=0).sort_values().index)

subsets = [reactionNames[0:13], reactionNames[13:27]]

In [ ]:
%%time

figs = []
ax = []

for subset in subsets:
    
    # reorder/subset columns by sorted medians
    coresamples_reorder = coresamples[subset]
    genomescalesampless_reorder = genomescalesamples[subset]
    
    # subset
    coresamplessubset = coresamples_reorder.iloc[randomindices,:]
    genomescalesamplessubset = genomescalesampless_reorder.iloc[randomindices,:]

    # melt and merge
    df1 = pd.melt(coresamplessubset, var_name='g', value_name='x')
    df2 = pd.melt(genomescalesamplessubset, var_name='g', value_name='x')
    merged = df1.merge(df2, on="g")
    
    # set category on data frame, to get reactions in desired order
    merged['g'] = merged['g'].astype(CategoricalDtype(subset))
    
    # set column labels
    merged.columns = ['g', 'Core Model', 'Genome Scale Model']
    
    # generate plot
    fig, axes = joyplot(
        merged, 
        by='g', 
        figsize=(9,9), 
        ylim='own', 
        legend=True, 
        ylabelsize=20, 
        xlabelsize=20, 
        color=['#1170aa', '#fc7d0b'],
        alpha=0.85,
    )
    
    # label the X axis
    axes[-1].set_xlabel('Flux (mmol/gCDW/h)', fontsize=20)
    
    figs.append(fig)
    ax.append(axes)

In [ ]:
figs[0].savefig('flux1.svg', format='svg', dpi=600)
figs[1].savefig('flux2.svg', format='svg', dpi=600)